In [10]:
import numpy as np
import scipy
import matplotlib
import seaborn
import pandas as pd
import altair
import vega_datasets
import sklearn
import bokeh
import datashader
import holoviews
import wordcloud
import spacy

### Import data

In [18]:
# Food expenditures data from ERS (Economic Research Service, USDA)
foodexp = pd.read_csv('faps_fahitem_puf.csv')
with open(, 'rb') as f:
  contents = f.read()
# NOTE - I need to find the documentation for this, not sure what dates I am looking at?

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 12: invalid start byte

In [12]:
# Monthly expenditures data, ERS (USDA) Updated Feb 25, 2020
#Note, 'constant dollar' refers to 1988 = 100
monthly_sales=pd.read_csv('monthly_sales_cleaned.csv')
monthly_sales.head()

,Year,Month,FAH_nominal_wtax_wtips_million,FAFH_nominal_wtax_wtips_million,Total_nominal_food_sales_million,FAH_constdollar_million,FAFH_constdollar_million,Total_constant_dollar_food_sales_million
0,2019.0,December,"74,901.67","73,337.91","148,239.58","36,125.06","31,012.62","67,423.64"
1,2019.0,November,"69,798.19","69,585.45","139,383.64","33,666.99","29,510.12","63,482.68"
2,2019.0,October,"67,301.77","71,609.53","138,911.30","32,380.60","30,417.63","63,227.35"
3,2019.0,September,"64,101.42","69,391.72","133,493.14","30,942.60","29,531.69","60,922.88"
4,2019.0,August,"69,333.52","75,675.25","145,008.77","33,522.32","32,289.19","66,314.81"


### EDA: monthly_sales

In [13]:
monthly_sales.shape
#There are 281 rows and 8 columns

(281, 8)

In [14]:
monthly_sales.dtypes.value_counts()
monthly_sales.dropna()

,Year,Month,FAH_nominal_wtax_wtips_million,FAFH_nominal_wtax_wtips_million,Total_nominal_food_sales_million,FAH_constdollar_million,FAFH_constdollar_million,Total_constant_dollar_food_sales_million
0,2019.0,December,"74,901.67","73,337.91","148,239.58","36,125.06","31,012.62","67,423.64"
1,2019.0,November,"69,798.19","69,585.45","139,383.64","33,666.99","29,510.12","63,482.68"
2,2019.0,October,"67,301.77","71,609.53","138,911.30","32,380.60","30,417.63","63,227.35"
3,2019.0,September,"64,101.42","69,391.72","133,493.14","30,942.60","29,531.69","60,922.88"
4,2019.0,August,"69,333.52","75,675.25","145,008.77","33,522.32","32,289.19","66,314.81"
5,2019.0,July,"68,024.06","74,865.75","142,889.81","32,861.14","32,012.86","65,380.20"
6,2019.0,June,"66,585.59","74,299.81","140,885.39","32,159.85","31,835.12","64,515.51"
7,2019.0,May,"69,079.83","75,131.65","144,211.48","33,262.84","32,296.27","66,026.86"
8,2019.0,April,"64,854.45","70,190.13","135,044.58","31,262.73","30,235.69","61,926.26"
9,2019.0,March,"66,186.03","72,894.99","139,081.02","31,815.57","31,502.33","63,773.25"


In [15]:
#Change dtypes from 'object' to float for other columns besides month

numcols=["FAH_nominal_wtax_wtips_million", 
               "FAFH_nominal_wtax_wtips_million", 
               "Total_nominal_food_sales_million", 
               "FAH_constdollar_million", 
               "FAFH_constdollar_million", 
               "Total_constant_dollar_food_sales_million"]

# Remove commas
monthly_sales[numcols] = monthly_sales[numcols].replace({',': ''}, regex=True)

#Convert strings without commas into numeric values
monthly_sales[monthly_sales.columns[2:]] = monthly_sales[monthly_sales.columns[2:]].apply(pd.to_numeric, downcast='float')
monthly_sales.dtypes.value_counts()

float32    6
float64    1
object     1
dtype: int64

In [16]:
monthly_sales.describe()

,Year,FAH_nominal_wtax_wtips_million,FAFH_nominal_wtax_wtips_million,Total_nominal_food_sales_million,FAH_constdollar_million,FAFH_constdollar_million,Total_constant_dollar_food_sales_million
count,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000
mean,2008.000000,48386.148438,43482.070312,91868.218750,27138.095703,24029.515625,51095.445312
std,6.645299,11113.704102,13977.619141,24894.390625,2800.712402,3553.622070,6306.088867
min,1997.000000,28154.230469,21313.570312,49467.800781,20533.080078,16687.750000,37353.718750
25%,2002.000000,38072.917969,31656.329590,69660.880859,25205.272949,21357.254883,46476.108398
50%,2008.000000,48919.115234,42002.826172,90843.816406,27158.325195,23785.455078,50804.230469
75%,2014.000000,57439.553711,53139.175781,111040.453125,28470.807129,26107.371582,55041.477539
max,2019.000000,74901.671875,75675.250000,148239.578125,36125.058594,32296.269531,67423.640625


The data spans 1997 to December 2019. 

FAH = "Food At Home"
FAFH = "Food Away From Home"

Does not includ schools/institutions. Data is in millions of US dollars. Values are total monthly sales.